# Foundations Of Computer Science Exam Project
## Team: Silvia Bordogna, Davide Brinati

Starting from the The Movie DataBase (TMDB) sample datasets,

1. For each movie, compute the number of cast members
2. How many movies do not have a homepage?
3. For each year, how many movies do not have a homepage?
4. Extract the domain of each homepage.
5. Extract a set of normalized tables. That is, each entry of a normalized table must contain exactly one value (not a list or a dictionary).
6. For each movie, compute the gross margin (difference between revenue and budget)
7. For each movie, compute the number of crew members
8. For each movie, compute the number of directors
9. For each language, compute the number of movies where such language is spoken.
10. For each company and each decade, compute the overall revenue
11. For each decade, compute the company with maximum revenue
12. In each year, how many movies have revenue smaller than the budget?

The following part of the exercise must be done only by groups of two or three people
1. Distribute the revenue according to the order of appearance in a movie. Assume that the i-th actor contributes twice as much as the (i+1)-th actor to the revenue.
2. For each actor find the total revenue attributed to him/her.
2. Find the actor that is responsible for the most overall revenue.


In [3]:
import pandas as pd
import re
import numpy as np

In [315]:
movies = pd.read_csv('tmdb_5000_movies.csv', index_col = 3)
#movies.head(3)
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 19995 to 25975
Data columns (total 19 columns):
budget                  4803 non-null int64
genres                  4803 non-null object
homepage                1712 non-null object
keywords                4803 non-null object
original_language       4803 non-null object
original_title          4803 non-null object
overview                4800 non-null object
popularity              4803 non-null float64
production_companies    4803 non-null object
production_countries    4803 non-null object
release_date            4802 non-null object
revenue                 4803 non-null int64
runtime                 4801 non-null float64
spoken_languages        4803 non-null object
status                  4803 non-null object
tagline                 3959 non-null object
title                   4803 non-null object
vote_average            4803 non-null float64
vote_count              4803 non-null int64
dtypes: float64(3), int64(3), obj

In [317]:
#we decide to REMOVE this movie because it does not have any information a part from title
movies[movies['release_date'].isnull()]

,budget,genres,homepage,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
id,,,,,,,,,,,,,,,,,,,
380097,0,[],NaN,[],en,America Is Still the Place,1971 post civil rights San Francisco seemed li...,0.0,[],[],NaN,0,0.0,[],Released,NaN,America Is Still the Place,0.0,0


In [318]:
#we decide to KEEP these movies because they have nearly all information a part from runtime
movies[movies['runtime'].isnull()] 

,budget,genres,homepage,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
id,,,,,,,,,,,,,,,,,,,
370980,15000000,"[{""id"": 18, ""name"": ""Drama""}]",NaN,"[{""id"": 717, ""name"": ""pope""}, {""id"": 5565, ""na...",it,Chiamatemi Francesco - Il Papa della gente,NaN,0.738646,"[{""name"": ""Taodue Film"", ""id"": 45724}]","[{""iso_3166_1"": ""IT"", ""name"": ""Italy""}]",2015-12-03,0,NaN,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,NaN,Chiamatemi Francesco - Il Papa della gente,7.3,12
459488,2,"[{""id"": 99, ""name"": ""Documentary""}]",NaN,"[{""id"": 6027, ""name"": ""music""}, {""id"": 225822,...",en,"To Be Frank, Sinatra at 100",NaN,0.050625,"[{""name"": ""Eyeline Entertainment"", ""id"": 60343}]","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""}]",2015-12-12,0,NaN,[],Released,NaN,"To Be Frank, Sinatra at 100",0.0,0


In [319]:
#remove movies without release date
movies.dropna(subset = ['release_date'], inplace = True) #remove movies without release date

In [320]:
credits = pd.read_csv('tmdb_5000_credits.csv', index_col = 0)
#credits.head()
credits.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 19995 to 25975
Data columns (total 3 columns):
title    4803 non-null object
cast     4803 non-null object
crew     4803 non-null object
dtypes: object(3)
memory usage: 150.1+ KB


### 1.1 For each movie, compute the number of cast members

In [321]:
# for each movie we count all unique id in cast members

#import time
#start_time = time.time()
cast_n = [(credits['title'].loc[i], len(pd.read_json(credits['cast'].loc[i])['id'].unique())) for i in credits.index if credits['cast'].loc[i] != '[]']
#print("--- %s seconds ---" % (time.time() - start_time))    
cast_n[1:10]

[("Pirates of the Caribbean: At World's End", 33),
 ('Spectre', 83),
 ('The Dark Knight Rises', 158),
 ('John Carter', 27),
 ('Spider-Man 3', 143),
 ('Tangled', 13),
 ('Avengers: Age of Ultron', 72),
 ('Harry Potter and the Half-Blood Prince', 49),
 ('Batman v Superman: Dawn of Justice', 152)]

In [48]:
#import time
#start_time = time.time()

#credits['n_cast']=credits.apply(lambda row: len(pd.read_json(row['cast'])['id'].unique()) if row['cast'] != '[]' else np.nan , axis=1)

#print("--- %s seconds ---" % (time.time() - start_time))   

#credits[['title','n_cast']].head()

--- 10.4839999676 seconds ---


### 1.2 How many movies do not have a homepage?

In [322]:
sum(pd.isnull(movies['homepage']))

3090

### 1.3 For each year, how many movies do not have a homepage?

In [359]:
#create year coloumn
movies['year'] = [date[0:4] for date in movies['release_date'] if pd.notnull(date)] 
#check if are there null homepage
movies_no_hp = movies.groupby(['year'])['homepage'].apply(lambda x: sum(x.isnull())) 

movies_no_hp.head()

year
1916    1
1925    1
1927    1
1929    2
1930    1
Name: homepage, dtype: int64

### 1.4 Extract the domain of each homepage.

In [5]:
#movies['homepage'].iloc[1]

In [6]:
#start_time = time.time()

[re.findall('://([\w+\.+\w+]*)/', i) for i in movies['homepage'] if pd.notnull(i)] #list comprehension with regex

#print("--- %s seconds ---" % (time.time() - start_time)) 

[['www.avatarmovie.com'],
 ['disney.go.com'],
 ['www.sonypictures.com'],
 ['www.thedarkknightrises.com'],
 ['movies.disney.com'],
 ['www.sonypictures.com'],
 ['disney.go.com'],
 ['marvel.com'],
 ['harrypotter.warnerbros.com'],
 ['www.batmanvsupermandawnofjustice.com'],
 [],
 ['www.mgm.com'],
 ['disney.go.com'],
 ['disney.go.com'],
 ['www.manofsteel.com'],
 ['marvel.com'],
 ['disney.go.com'],
 ['www.sonypictures.com'],
 ['www.thehobbit.com'],
 [],
 ['www.robinhoodthemovie.com'],
 ['www.thehobbit.com'],
 ['www.goldencompassmovie.com'],
 [],
 ['marvel.com'],
 ['www.jurassicworld.com'],
 [],
 ['www.sonypictures.com'],
 ['marvel.com'],
 ['disney.go.com'],
 ['www.transformersmovie.com'],
 [],
 [],
 [],
 ['disney.go.com'],
 ['www.disney.go.com'],
 ['greenlanternmovie.warnerbros.com'],
 ['disney.go.com'],
 ['www.warnerbros.com'],
 ['www.furious7.com'],
 [],
 [],
 [],
 ['disney.go.com'],
 ['www.pacificrimmovie.com'],
 ['www.transformersmovie.com'],
 ['www.indianajones.com'],
 ['movies.disney.co

### 1.5 Extract a set of normalized tables. That is, each entry of a normalized table must contain exactly one value (not a list or a dictionary).

In [352]:
#create function to normalize each json coloumn
def normalize_json(df, variable_name):
    variable_list = []
    for i in df.index:
        df_temp = pd.read_json((df[variable_name].loc[i]))
        if not df_temp.empty:
            variable_list.append(df_temp) 
    return pd.concat(variable_list, axis = 0, ignore_index = True).drop_duplicates().reset_index(drop = True)

In [396]:
cast = normalize_json(df = credits, variable_name = 'cast')
#extract just cast specific details
cast[['cast_id','character','credit_id','order']].drop_duplicates().reset_index(drop = True).head()

,cast_id,character,credit_id,order
0,242,Jake Sully,5602a8a7c3a3685532001c9a,0
1,3,Neytiri,52fe48009251416c750ac9cb,1
2,25,Dr. Grace Augustine,52fe48009251416c750aca39,2
3,4,Col. Quaritch,52fe48009251416c750ac9cf,3
4,5,Trudy Chacon,52fe48009251416c750ac9d3,4


In [394]:
crew = normalize_json(df = credits, variable_name = 'crew')
#extract just crew specific details
crew[['credit_id','department','job']].drop_duplicates().reset_index(drop = True).head()

,credit_id,department,job
0,52fe48009251416c750aca23,Editing,Editor
1,539c47ecc3a36810e3001f87,Art,Production Design
2,54491c89c3a3680fb4001cf7,Sound,Sound Designer
3,54491cb70e0a267480001bd0,Sound,Supervising Sound Editor
4,539c4a4cc3a36810c9002101,Production,Casting


In [391]:
#create table with information related to persons shared by both cast and crew
people=pd.concat([crew[['id','name','gender']], cast[['id','name','gender']]]).drop_duplicates().reset_index(drop = True).head()
people.head()

,id,name,gender
0,1721,Stephen E. Rivkin,0
1,496,Rick Carter,2
2,900,Christopher Boyes,0
3,1262,Mali Finn,1
4,1729,James Horner,2


In [100]:
spoken_languages = normalize_json(df = movies, variable_name = 'spoken_languages')
spoken_languages.head()

,iso_639_1,name
0,en,English
1,es,Español
2,fr,Français
3,it,Italiano
4,de,Deutsch


In [99]:
genres = normalize_json(df = movies, variable_name = 'genres')
genres.head()

,id,name
0,28,Action
1,12,Adventure
2,14,Fantasy
3,878,Science Fiction
4,80,Crime


In [98]:
production_countries = normalize_json(df = movies, variable_name = 'production_countries')
production_countries.head()

,iso_3166_1,name
0,US,United States of America
1,GB,United Kingdom
2,JM,Jamaica
3,BS,Bahamas
4,DM,Dominica


In [97]:
production_companies = normalize_json(df = movies, variable_name = 'production_companies')
production_companies.head()

,id,name
0,289,Ingenious Film Partners
1,306,Twentieth Century Fox Film Corporation
2,444,Dune Entertainment
3,574,Lightstorm Entertainment
4,2,Walt Disney Pictures


In [96]:
keywords = normalize_json(df = movies, variable_name = 'keywords')
keywords.head()

,id,name
0,1463,culture clash
1,2964,future
2,3386,space war
3,3388,space colony
4,3679,society


### 1.6 For each movie, compute the gross margin (difference between revenue and budget)

In [360]:
movies['gross_margin'] = movies['revenue'] - movies['budget'] 
movies[['title','gross_margin']].sort_values(['gross_margin'], ascending=False).head()

,title,gross_margin
id,,
19995,Avatar,2550965087
597,Titanic,1645034188
135397,Jurassic World,1363528810
168259,Furious 7,1316249360
24428,The Avengers,1299557910


### 1.7 For each movie, compute the number of crew members

In [361]:
crew_n = [(credits['title'].loc[i], len(pd.read_json(credits['crew'].loc[i])['id'].unique())) for i in credits.index if credits['crew'].loc[i] != '[]']
crew_n[0:10]

[('Avatar', 146),
 ("Pirates of the Caribbean: At World's End", 32),
 ('Spectre', 150),
 ('The Dark Knight Rises', 210),
 ('John Carter', 129),
 ('Spider-Man 3', 54),
 ('Tangled', 76),
 ('Avengers: Age of Ultron', 71),
 ('Harry Potter and the Half-Blood Prince', 27),
 ('Batman v Superman: Dawn of Justice', 109)]

In [105]:
#for i in credits.index:
#    temp=pd.read_json((credits['crew'].loc[i]))
#    if not temp.empty:
#        print(credits['title'].loc[i], len(temp['id'].unique()))

### 1.8 For each movie, compute the number of directors

In [341]:
director_n = [(credits['title'].loc[i], sum(pd.read_json((credits['crew'].loc[i]))['job'] == 'Director')) for i in credits.index if credits['crew'].loc[i] != '[]']
director_n[0:10]

[('Avatar', 1),
 ("Pirates of the Caribbean: At World's End", 1),
 ('Spectre', 1),
 ('The Dark Knight Rises', 1),
 ('John Carter', 1),
 ('Spider-Man 3', 1),
 ('Tangled', 2),
 ('Avengers: Age of Ultron', 1),
 ('Harry Potter and the Half-Blood Prince', 1),
 ('Batman v Superman: Dawn of Justice', 1)]

In [111]:
#for i in credits.index:
#    title = credits['title'].loc[i]
#    sum_director = sum(pd.read_json((credits['crew'].loc[i]))['job'] == 'Director') #read json extract job and create boolean T/F
#    print(title, sum_director)

### 1.9 For each language, compute the number of movies where such language is spoken.

In [344]:
language = []

for i in movies.index:
    temp = pd.read_json((movies['spoken_languages'].loc[i]))
    temp['title'] = movies['title'].loc[i]
    language.append(temp)
    
language = pd.concat(language, axis = 0, ignore_index = True)
#language.head()

In [345]:
language.groupby(['iso_639_1'])['title'].count().head()

iso_639_1
af     7
am     1
ar    67
bg     3
bm     1
Name: title, dtype: int64

### 1.10 For each company and each decade, compute the overall revenue

In [362]:
#create decade coloumn
movies['decade'] = [str(date[0:3]) + '0' for date in movies['release_date'] if pd.notnull(date)] 

#per each movie extract company and split revnues  
company = []
for i in movies.index:
    temp = pd.read_json((movies['production_companies'].loc[i]))
    if not temp.empty:
        temp['movie_id'] = i
        temp['decade'] = movies['decade'].loc[i]
        temp['revenue'] = movies['revenue'].loc[i]/temp.shape[0] #we assume each company earn same share of revenue 
        company.append(temp)

company = pd.concat(company, axis = 0, ignore_index = True)

In [367]:
# sum revenues per decade company
rev_company_per_decade = company.groupby(['name','decade'])['revenue'].sum().reset_index()
rev_company_per_decade.head()

,name,decade,revenue
0,"""DIA"" Productions GmbH & Co. KG",2000,11087731
1,1.85 Films,2010,12252
2,10 West Studios,2010,0
3,100 Bares,2000,3773982
4,100 Bares,2010,2666666


### 1.11 For each decade, compute the company with maximum revenue

In [354]:
# compute max value per each decade
max_value = rev_company_per_decade.reset_index().groupby(['decade'])['revenue'].max()
max_value.rename("decade_revenue", inplace = True)

decade
1910       4197375
1920      26358000
1930     200088229
1940     309597150
1950      68500000
1960     307234055
1970     767330336
1980    3082731008
1990    4602361219
2000    6133589420
2010    6190159565
Name: decade_revenue, dtype: int64

In [355]:
#use join to get all company with max value
join = rev_company_per_decade.join(max_value, on = 'decade')
join[join['revenue'] == join['decade_revenue']].sort_values(['decade'])

,name,decade,revenue,decade_revenue
5555,Triangle Film Corporation,1910,4197375,4197375
5872,Wark Producing Corp.,1910,4197375,4197375
3472,Metro-Goldwyn-Mayer (MGM),1920,26358000,26358000
3473,Metro-Goldwyn-Mayer (MGM),1930,200088229,200088229
4773,Selznick International Pictures,1930,200088229,200088229
5858,Walt Disney Productions,1940,309597150,309597150
5599,Twentieth Century Fox Film Corporation,1950,68500000,68500000
1705,Eon Productions,1960,307234055,307234055
5689,Universal Pictures,1970,767330336,767330336
4079,Paramount Pictures,1980,3082731008,3082731008


### 1.12 In each year, how many movies have revenue smaller than the budget?

In [368]:
movies[movies['revenue'] < movies['budget']].groupby(['year'])['title'].count().tail(10) 

year
2007    55
2008    64
2009    78
2010    65
2011    70
2012    52
2013    62
2014    59
2015    67
2016    26
Name: title, dtype: int64

### 1.13 Distribute the revenue according to the order of appearance in a movie. Assume that the i-th actor contributes twice as much as the (i+1)-th actor to the revenue.

In [373]:
actors_revenue_list = []

for i in movies.index:
    cast=pd.read_json((credits['cast'].loc[i]))
    
    if not cast.empty and movies['revenue'].loc[i]>0  :
        
        assert movies['revenue'].loc[i] > 0.0, 'Revenues for %s th movie are 0' %i
        
        x = float(float(movies['revenue'].loc[i]) / float(2**len(cast) - 1))
        n_actors=len(cast)
        
        for j in range(0, n_actors):
            
            exponent = n_actors - (cast['order'][j])
            rev = int(x*float(2**float(exponent)))
            actors_revenue_list.append((movies['title'].loc[i], cast['name'][j], cast['id'][j], cast['order'][j], rev))

actor_revenue=pd.DataFrame(actors_revenue_list, columns=('movie_title','name','id','order','revenue'))

In [374]:
actor_revenue.head()

,movie_title,name,id,order,revenue
0,Avatar,Sam Worthington,65731,0,2787965087
1,Avatar,Zoe Saldana,8691,1,1393982543
2,Avatar,Sigourney Weaver,10205,2,696991271
3,Avatar,Stephen Lang,32747,3,348495635
4,Avatar,Michelle Rodriguez,17647,4,174247817


### 1.14 For each actor find the total revenue attributed to him/her.

In [377]:
tot_rev_actor=actor_revenue.groupby(['name'])['revenue'].sum().sort_values(ascending=False)
tot_rev_actor.head()

name
Tom Cruise           7952239511
Tom Hanks            7600248675
Robert Downey Jr.    7263478985
Johnny Depp          7023041802
Will Smith           6563196471
Name: revenue, dtype: int64

### 1.15 Find the actor that is responsible for the most overall revenue.

In [378]:
np.argmax(tot_rev_actor)

u'Tom Cruise'